# **Preprocess**

In [11]:
import pandas as pd
http = pd.read_csv("C:\\Users\\pranil\\Desktop\\burpy-master\\merge.csv")

In [4]:
!pip install pycaret

In [12]:
from pycaret.clustering import *
clu1 = setup(data = http, normalize = True,numeric_features = ['single_q','double_q','dashes','braces','spaces','badwords'],ignore_features = ['method','path','body','class'])


,Description,Value
0,Session id,3144
1,Original data shape,"(2342, 10)"
2,Transformed data shape,"(2342, 2046)"
3,Ignore features,4
4,Numeric features,6
5,Categorical features,3
6,Preprocess,True
7,Imputation type,simple
8,Numeric imputation,mean
9,Categorical imputation,mode


In [13]:
kmeans = create_model('kmeans', num_clusters = 2)

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.0486,2.4338,3.6227,0,0,0


Processing:   0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
kmeans

KMeans(n_clusters=2, random_state=3144)

In [15]:
plot_model(kmeans)

In [ ]:
plot_model(kmeans, plot = 'tsne')

In [16]:
kmeans_result = assign_model(kmeans)

In [17]:
merged_data = pd.concat([http, kmeans_result[['Cluster']]], axis=1)

# Save the merged data to CSV including the 'class' column
#merged_data.to_csv('/kmeans_result_with_class.csv', index=False)

In [18]:
import pandas as pd
test_http = pd.read_csv("C:\\Users\\pranil\\Desktop\\burpy-master\\test.csv")
test_http.head()

,Method,Path,Body,single_q,double_q,dashes,braces,spaces,badwords
0,POST,b'/openrtb2/auction',"b'{""imp"":[{""ext"":{""tid"":""44e8281d-cf19-4593-a8...",0,212,1,2,11,0
1,POST,b'/header/auction?lib=prebid&v=7.50.0&referrer...,"b'{""imp"":[{""id"":0,""tagid"":""gfg_desktop_300x250...",0,298,1,0,0,0
2,POST,b'/hb/bid-request',"b'{""referrer"":""https://www.geeksforgeeks.org/d...",0,106,1,0,0,0
3,POST,b'/hb-mm-multi',"b'{""params"":{""wrapper_type"":""prebidjs"",""wrappe...",0,124,1,2,11,0
4,POST,b'/openrtbb/prebidjs',"b'{""imp"":[{""ext"":{""tid"":""44e8281d-cf19-4593-a8...",0,194,1,2,11,0


In [19]:
pip install pycaret

Note: you may need to restart the kernel to use updated packages.


In [20]:
from pycaret import *
from pycaret.clustering import predict_model

# 'test_http' is your dataset for prediction
test_result = predict_model(kmeans, data=test_http)

In [21]:
test_result.to_csv("/test_results")


PermissionError: [Errno 13] Permission denied: '/test_results'

In [22]:

from http.server import *
from urllib import request, error
import pandas as pd
from pycaret.clustering import*
import urllib.parse
import sys

In [23]:
badwords = ['sleep','drop','uid','select','waitfor','delay','system','union','order by','group by']
def ExtractFeatures(path):
  path = urllib.parse.unquote(path)
  badwords_count = 0
  single_q = path.count("'")
  double_q = path.count("\"")
  dashes = path.count("--")
  braces = path.count("(")
  spaces = path.count(" ")
  for word in badwords:
    badwords_count += path.count(word)
  lst = [single_q,double_q,dashes,braces,spaces,badwords_count]
  print(lst)
  return pd.DataFrame([lst],columns = ['single_q','double_q','dashes','braces','spaces','badwords'])

In [24]:
http = pd.read_csv("C:\\Users\\pranil\\Desktop\\burpy-master\\merge.csv")
clu1 = setup(data = http, normalize = True,numeric_features = ['single_q','double_q','dashes','braces','spaces','badwords'] , ignore_features = ['method','path','body','class'] )
kmeans = create_model('kmeans',num_clusters = 2)

,Description,Value
0,Session id,2489
1,Original data shape,"(2342, 10)"
2,Transformed data shape,"(2342, 2046)"
3,Ignore features,4
4,Numeric features,6
5,Categorical features,3
6,Preprocess,True
7,Imputation type,simple
8,Numeric imputation,mean
9,Categorical imputation,mode


,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.0931,5.4112,13.5754,0,0,0


Processing:   0%|          | 0/3 [00:00<?, ?it/s]

In [27]:
from http.server import SimpleHTTPRequestHandler, HTTPServer
from urllib import request, error
import sys

# Assuming ExtractFeatures is defined and functional
# Also assuming a KMeans model named 'kmeans' is trained previously

class SimpleHTTPProxy(SimpleHTTPRequestHandler):
    proxy_routes = {}

    @classmethod
    def set_routes(cls, proxy_routes):
        cls.proxy_routes = proxy_routes

    def do_GET(self):
        parts = self.path.split('/')
        print(parts)

        if len(parts) >= 4:
            live_data = ExtractFeatures(parts[3])  # Fixed typo: parts, not part
            result = predict_model(kmeans, data=live_data)
            print(result['Cluster'][0])
            if result['Cluster'][0] == "Cluster 1":
                print('Intrusion Detected')

        if len(parts) >= 3:
            self.proxy_request('http://' + parts[2] + '/')
        else:
            super().do_GET()

    def proxy_request(self, url):
        try:
            response = request.urlopen(url)
        except error.HTTPError as e:
            print('err')
            self.send_response_only(e.code)
            self.end_headers()
            return

        self.send_response_only(response.status)
        for name, value in response.headers.items():
            self.send_header(name, value)
        self.end_headers()
        self.copyfile(response, self.wfile)

SimpleHTTPProxy.set_routes({'proxy_route': 'http://demo.testfire.net/'})
with HTTPServer(('127.0.0.1', 21), SimpleHTTPProxy) as httpd:
    host, port = httpd.socket.getsockname()
    print(f'Listening on http://{host}:{port}')
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        print("\nKeyboard interrupt received, exiting")
        sys.exit(0)


Listening on http://127.0.0.1:21
['http:', '', 'demo.testfire.net', '']
[0, 0, 0, 0, 0, 0]
['http:', '', 'demo.testfire.net', '']
[0, 0, 0, 0, 0, 0]
['http:', '', 'demo.testfire.net', '']
[0, 0, 0, 0, 0, 0]
['http:', '', 'demo.testfire.net', '']
[0, 0, 0, 0, 0, 0]


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 55616)
Traceback (most recent call last):
  File "c:\Users\pranil\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "c:\Users\pranil\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "c:\Users\pranil\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "c:\Users\pranil\AppData\Local\Programs\Python\Python39\lib\http\server.py", line 653, in __init__
    super().__init__(*args, **kwargs)
  File "c:\Users\pranil\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 720, in __init__
    self.handle()
  File "c:\Users\pranil\AppData\Local\Programs\Python\Python39\lib\http\s

['http:', '', 'demo.testfire.net', '']
[0, 0, 0, 0, 0, 0]
['http:', '', 'demo.testfire.net', '']
[0, 0, 0, 0, 0, 0]
['http:', '', 'demo.testfire.net', '']
[0, 0, 0, 0, 0, 0]
['http:', '', 'demo.testfire.net', '']
[0, 0, 0, 0, 0, 0]


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 55620)
Traceback (most recent call last):
  File "c:\Users\pranil\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "c:\Users\pranil\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "c:\Users\pranil\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "c:\Users\pranil\AppData\Local\Programs\Python\Python39\lib\http\server.py", line 653, in __init__
    super().__init__(*args, **kwargs)
  File "c:\Users\pranil\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 720, in __init__
    self.handle()
  File "c:\Users\pranil\AppData\Local\Programs\Python\Python39\lib\http\s

['http:', '', 'demo.testfire.net', '']
[0, 0, 0, 0, 0, 0]
['http:', '', 'demo.testfire.net', '']
[0, 0, 0, 0, 0, 0]


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 55624)
Traceback (most recent call last):
  File "c:\Users\pranil\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "c:\Users\pranil\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "c:\Users\pranil\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "c:\Users\pranil\AppData\Local\Programs\Python\Python39\lib\http\server.py", line 653, in __init__
    super().__init__(*args, **kwargs)
  File "c:\Users\pranil\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 720, in __init__
    self.handle()
  File "c:\Users\pranil\AppData\Local\Programs\Python\Python39\lib\http\s


Keyboard interrupt received, exiting


AssertionError: 